# API Keys

Export NVIDIA API keys for authentication
Get your API keys from: https://build.nvidia.com

# Start Services

Start all required services for CA-RAG:
1. Export your NVIDIA API key:
   ```
   export NVIDIA_API_KEY=xxxx
   ```

2. Start the services using docker-compose:
   ```
   make -C docker start_compose
   ```

This will start the ingestion service (default port 8001) and retrieval service (default port 8000).

In [ ]:
%env NVIDIA_BUILD_API_KEY=xxxx
%env NVIDIA_API_KEY=xxxx

# Install NV-Ingest

Install NV-Ingest following the steps

Create a new uv virtual environment with nv-ingest

```
uv venv --python 3.12 nvingest && \
  source nvingest/bin/activate && \
  uv pip install pypdfium2==4.30.1
  uv pip install nv-ingest==25.6.2 nv-ingest-api==25.6.2 nv-ingest-client==25.6.3
```
For more details, see nvingest documentation [here](https://docs.nvidia.com/nemo/retriever/latest/extraction/quickstart-library-mode/)

# Start NV-Ingest Client

Start the nv-ingest client to process the pdf documents

In [ ]:
from nv_ingest.framework.orchestration.ray.util.pipeline.pipeline_runners import run_pipeline
from nv_ingest.framework.orchestration.ray.util.pipeline.pipeline_runners import PipelineCreationSchema
from nv_ingest_client.client import Ingestor, NvIngestClient
from nv_ingest_api.util.message_brokers.simple_message_broker import SimpleClient
from nv_ingest_client.util.process_json_files import ingest_json_results_to_blob
# Start the pipeline subprocess for library mode
config = PipelineCreationSchema()

run_pipeline(config, block=False, disable_dynamic_scaling=True, run_in_subprocess=True)

client = NvIngestClient(
    message_client_allocator=SimpleClient,
    message_client_port=7670,
    message_client_hostname="localhost"
)

In [ ]:
ingestor = (
    Ingestor(client=client)
    .files("data/multimodal_test.pdf")
    .extract(
        extract_text=True,
        extract_tables=True,
        extract_charts=True,
        extract_images=False,
        paddle_output_format="markdown",
        extract_infographics=True,
        # extract_method="nemoretriever_parse", #Slower, but maximally accurate, especially for PDFs with pages that are scanned images
        text_depth="page"
    )
)

results, failures = ingestor.ingest(show_progress=True, return_failures=True)

print(ingest_json_results_to_blob(results[0]))
print(failures)


# Ingest the documents in Context Aware RAG

### Add Documents to Context-Aware RAG

This section demonstrates how to add documents to CA-RAG using POST requests to the ingestion service. We'll upload the previously extracted text, tables and charts for data ingestion. Once ingestion is complete, the documents will be available for question-answering.

In [ ]:
# Upload text results to server
import requests
import json
import pyaml_env

ingestion_url = "http://<HOST>:<PORT>" # default is localhost:8001
headers = {"Content-Type": "application/json"}

config = pyaml_env.parse_config("../config/config.yaml")

# Initialize service
init_data = {"context_config": config, "uuid": "1"}
response = requests.post(
    f"{ingestion_url}/init", headers=headers, data=json.dumps(init_data)
)

# Extract text documents from results
add_doc_data_list = []
doc_index = 0
for _, doc in enumerate(results[0]):
    if doc["document_type"] == "text":
        doc_data = {
            "document": doc["metadata"]["content"],
            "doc_index": doc_index
        }
        
        # Add metadata for first/last docs
        if doc_index == 0:
            doc_data["doc_metadata"] = {"is_first": True}
            
        add_doc_data_list.append(doc_data)
        doc_index += 1

# Upload documents
for add_doc_data in add_doc_data_list:
    response = requests.post(
        f"{ingestion_url}/add_doc", headers=headers, data=json.dumps(add_doc_data)
    )
    print(f"Added document {add_doc_data['doc_index']}")

# Add the terminating document
doc_data = {
            "document": ".",
            "doc_index": doc_index,
            "doc_metadata": {"is_last": True}
        }
response = requests.post(
    f"{ingestion_url}/add_doc", headers=headers, data=json.dumps(doc_data)
)


# Q&A

# Retrieval Service
In this section, we initialize the retrieval service with the same UUID used for data ingestion. We then send a request to summarize the document using the retrieval service's API endpoints. The service processes the question and returns a summary based on the previously ingested documents.


In [ ]:
retrieval_url = "http://<HOST>:<PORT>" # default is http://localhost:8000

# Initialize service with same uuid as data ingestion

init_data = {"context_config": config, "uuid": "1"}


response = requests.post(
    f"{retrieval_url}/init", headers=headers, data=json.dumps(init_data)
)

# Send retrieval request
call_data = {"retriever_function": {"question": "Summarize the document."}}
request_data = {"state": call_data}
response = requests.post(
    f"{retrieval_url}/call", headers=headers, data=json.dumps(request_data)
)
print(response.json()["result"])